### Luigi Talamo  

Language Science and Technology

Saarland University (Germany)

luigi.talamo@uni-saarland.de

Code available from https://github.com/rahonalab/dhw-1/

Check also these scripts: https://github.com/rahonalab/doing_stuff_with_ciepplus which are used for actual papers !

# Experimenting with miniciep+

Welcome to this interactive Jupyter notebook!

Now that you know the story of miniciep+, let's start writing some code to interact with this wonderful parallel corpus.

## Python libraries for working with the CoNNL-U format
There are at least three libraries for working with the CoNLL-U format:

* conllu: https://pypi.org/project/conllu/
* pyconll: https://pyconll.readthedocs.io/en/stable/
* udapi: https://udapi.readthedocs.io/

I used to employ the first two libraries, then Andy told me about udapi.
Udapi is unfortunately poorly documented, but is a very powerful library. With the present tutorial, I hope to contribute a bit to its documentation. If you are looking at this file with Google colab, you don't need to install anything on your machine.
Later, when you will use Udapi locally, you will have to install the library with:

`pip3 install udapi`

or

`conda install udapi`


## First steps
First, let's import the udapi library in our Python script and copy the conllu files on our colab:

In [ ]:
!pip install udapi
import udapi
!wget https://raw.githubusercontent.com/rahonalab/dhw-1/main/dataset/conllu/it_nomerosa.conllu
!wget https://raw.githubusercontent.com/rahonalab/dhw-1/main/dataset/conllu/ro_nomerosa.conllu
!wget https://raw.githubusercontent.com/rahonalab/dhw-1/main/dataset/conllu/es_nomerosa.conllu
!wget https://raw.githubusercontent.com/rahonalab/dhw-1/main/dataset/conllu/pt_nomerosa.conllu
!wget https://raw.githubusercontent.com/rahonalab/dhw-1/main/dataset/conllu/fr_nomerosa.conllu

then, you can load a conllufile with the ``udapi.Document`` function. In the root folder, you find five connlu files containing the first sentences of Eco's Il nome della Rosa, in the original Italian version and in four Romance translations. Let's load the original Italian version:

In [ ]:
nomerosa = udapi.Document("it_nomerosa.conllu")

## Walk through sentences and tokens of a CoNNL-U file
Each udapi object like our ``it_nomerosa`` contains a list of sentences, which are callable through its ``.bundle`` attribute.
In turn, each sentence is a list of tokens, which are callable through its ``.nodes`` attribute.
There are basically two ways of exploring and collecting data from a sentence:
* a linear walk: we walk through the sentence following the linear ordering of the tokens (i.e., a flat list of tokens);
* a tree walk, we traverse the sentence using the tree structure described by the dependency relation (i.e. a nested tree structure of tokens).

With respect to other libraries, the same udapi object allows for both ways of exploring.

### Linear walk
Let's start with the linear walk, printing all tokens from the file:

In [ ]:
for sentence in nomerosa.bundles:
    for token in sentence.nodes:
        # Print tokens
        print(token)

Our library stores each of the UD annotation fields as an attribute of the object, e.g. `token.form` contains the graphemic form of the token.
These fields are: ``ord``, ``form``, ``lemma``, ``upos``, ``xpos``, ``feats``, ``parent``, ``deprel``, ``deps``, ``misc``.
Remember the fields we saw in the slides? We can loop over the CoNNL-U file and print them. For instance, let's print the four fields we have discussed in the slides, plus the ID (position number: ``ord``) and the graphemic form of the token:

In [ ]:
for sentence in nomerosa.bundles:
    for token in sentence.nodes:
        # Print tokens
        print("Token no. "+str(token.ord)+" has form "+str(token.form)+" UPOS: "+str(token.upos)+", FEATS: "+str(token.feats)+", DEPREL:"+str(token.deprel)+" and its head is token "+str(token.parent))

We can easily traverse the CoNNL-U file and collect simple statistics like 'How many verbs are there in the corpus?' or more complex ones like 'How many personal pronouns do behave as direct objects?'

In [ ]:
verb = 0
pron_obj = 0
prons = []
for sentence in nomerosa.bundles:
        for token in sentence.nodes:
            if token.upos == 'VERB':
                verb += 1
            if token.upos == "PRON" and token.deprel == "obj" and "Prs" in token.feats["PronType"]:
                pron_obj += 1
                prons.append(token.form)
print("There are "+str(verb)+" verbs."+" There are also "+str(pron_obj)+" personal pronouns behaving as direct objects. They are "+str(prons))

### Tree walk
To traverse the CoNLL-U file in a hierarchical way, we just have to use the ``children`` statement

In [ ]:
for sentence in nomerosa.bundles:
    for token in sentence.nodes:
        print(str(token.form)+" has the following dependencies "+str(token.children))


Let's write now a simple script to compute the word order patterns of adjectives, relative clauses and determiners across our five languages. First we loop over the folder where the CoNLL-U files are stored and load them as ``udapi`` object:

In [ ]:
import glob
for conllufile in sorted(glob.glob("*.conllu")):
    ud_doc = udapi.Document(conllufile)

Then, we traverse the CoNLL-U files in a linear way, and we apply a very simple algorithm:
* check whether the token is a NOUN (we use UPOS);
* if yes, we determine if it's modified by an adjective, a relative clause and a determiner (we use deprel);
* then, determine the word order: if the id of the head is higher than the id of the modifier, then we have a mod-head pattern, otherwise we have a head-mod pattern.
We set a python dictionary, ``results``, to take note of the results.

In [ ]:
import glob
import pprint #better printing
results = {}
for conllufile in sorted(glob.glob("*.conllu")):
    results[conllufile] = {
    "adjectives": {
        "h-m": int(0),
        "m-h": int(0)
    },
    "relative clauses": {
        "h-m": int(0),
        "m-h": int(0)
    },
    "determiners": {
        "h-m": int(0),
        "m-h": int(0)
    }
    }
    nomerosa = udapi.Document(conllufile)
    for sentence in nomerosa.bundles:
        for token in sentence.nodes:
            if token.upos == 'NOUN':
                for mod in token.children: #search through modifiers
                    if mod.deprel == "amod": #do we have an adjectival modification?
                        if token.ord < mod.ord:
                            results[conllufile]["adjectives"]["h-m"] += 1
                        else:
                            results[conllufile]["adjectives"]["m-h"] += 1
                    if mod.deprel in ["acl:relcl","acl"]: #do we have a modifier acting as a relative clause?
                        if token.ord < mod.ord:
                            results[conllufile]["relative clauses"]["h-m"] += 1
                        else:
                            results[conllufile]["relative clauses"]["m-h"] += 1
                    if mod.deprel in ["det"]: #do we have determiners (very broad category here, it could be articles, demonstratives, quantifiers,...)?
                        if token.ord < mod.ord:
                            results[conllufile]["determiners"]["h-m"] += 1
                        else:
                            results[conllufile]["determiners"]["m-h"] += 1
pprint.pprint(results)
